Данный файл служит для проверки работы обученной нейронной сети. Базируется на OCR Tesseract. Предварительно необходимо запустить файл create_model. 

In [1]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf

import pytesseract
from pytesseract import Output
from conf import *

Следующие две функции необходимы для обработки блоков полученных от программы tesseract. Данные переводятся в тот формат, в котором работает обученная модель.

In [2]:
def img_conversion(path, src=None):
    # Подгон под высоту
    if src is None:
        src = cv2.imread(path)
    size_h, size_w, _ = src.shape
    a = H_SIZE/size_h
    size = (round(size_w*a),H_SIZE)
    result2 = cv2.resize(src, size)
    
    img = Image.fromarray(result2)
    img = ImageOps.invert(img)
    size_w, size_h = img.size
    crop_image = img.crop((size_w-MEAN_W_SIZE_WORD, 0, size_w, size_h))
    
    img_rez = ImageOps.invert(crop_image).convert('L')
    row = np.reshape(img_rez, M)/255
    return (row, img_rez)

def cropped_img(img_word):
    img = 255 -img_word

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU, cv2.THRESH_BINARY_INV)

    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 2)

    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    img2 = img.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)     
        cropped = img2[y:y + h, x:x + w]

    img_crop = 255 - img2
    return img_crop, (x, y, w, h)

In [3]:
# Загрузка модели
model = tf.keras.models.load_model(NAME_MODEL)


name_file - путь к файлу, который будет обрабатываться. Полученные данные из tesseract подготавливаются, а затем обрабатываются в зависимости от того, к какому классу распределит модель данные. Результат выводится в качестве изображения.

In [4]:
name_file = f"../img/text_8.png"
img = cv2.imread(name_file)

d = pytesseract.image_to_data(img, config=CONFIG_TESSERACT, output_type=Output.DICT)
n_boxes = len(d['text'])
rows = np.zeros((n_boxes, M))
k = 0

for i in range(n_boxes):
    if int(d['conf'][i]) > CONF:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        if w>MIN_W and h>MIN_H:
            img_word = img[y:y+h, x:x+w, :]
            img_word_crop, delta = cropped_img(img_word)
            r, _ = img_conversion(None, img_word_crop)
            rows[k, :] = r
            k += 1

predict_model = model.predict(rows)   
k = 0

for i in range(n_boxes):
    if int(d['conf'][i]) > CONF:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        if w>MIN_W and h>MIN_H:
            img_word = img[y:y+h, x:x+w, :]
            img_word_crop, delta = cropped_img(img_word)
            r, im = img_conversion(None, img_word_crop)
            if MY_CROP:
                x = x+delta[0]
                y = y+delta[1]
                w = delta[2]
                h = delta[3]
            if predict_model[k].argmax() == 1:       
                img = cv2.rectangle(img, (x, y), (x + w, y + h), BOLD_COLOR, 2)
            elif predict_model[k].argmax() == 2:
                img = cv2.rectangle(img, (x, y), (x + w, y + h), ITALIC_COLOR, 2)
            else:
                img = cv2.rectangle(img, (x, y), (x + w, y + h), REGULAR_COLOR, 2)
            k += 1
cv2.namedWindow(NAME_WINDOW_IMG, cv2.WINDOW_NORMAL)
cv2.resizeWindow(NAME_WINDOW_IMG, SIZE_W_WINDOW, SIZE_H_WINDOW)
cv2.imshow(NAME_WINDOW_IMG, img)
cv2.waitKey(0)

14/14 [==============================] - 0s 2ms/step


-1